Langchain  is a python framework that wires multiple steps together while building an LLM application:

Components:

1. LLM: The language model you use to generate answers like a human and to understand the question.

2. Tools: These allow the LLM to call functions (tools) when needed via agents.

3. Agents: A smart system that knows which tool should be used.

4. vector database: perform semantic search for retrieving relevant documents or information.

In [ ]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os

LangChain runs your app — LangSmith watches and explains how. Gives you an overall view of the full process inside your LangChain application:

User Input
See exactly what the user asked.

LLM Interpretation
How the language model understood the question.

Agent Decisions
Which tool the agent chose to use (and why).

Tool Calls
What tools were called, with what parameters (e.g., employee_id = "123").

Tool Responses
The data returned from each tool/function.

Final LLM Response
The final answer generated using the tool's result.

Execution Timeline
Visual flow of steps: input → tool → output → answer.

🧪 Bonus: LangSmith Also Helps With
Debugging errors

Testing prompts or tool logic

Improving agent reasoning

Comparing different models or chain designs


In [ ]:
# Load environment variables from .env file
os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN_API_KEY" #get from .env api key for langsmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "HR assistant"

1. Tools:

Are functions or external capabilities (e.g., databases, APIs).

Perform specific tasks like querying a database, making a calculation, or generating content.

Do not decide which task to perform. They simply execute the action they're called to do.

2. Agents:

Are decision-makers.

Decide when and which tools to call based on the input (query) they receive.

They orchestrate the entire process of interacting with tools to achieve a goal.

In [ ]:
load_dotenv()
apikey = os.getenv('OPENAI_API_KEY')#load api key from .env

In [ ]:
# Set up LLM (OpenAI gpt-3.5-turbo)
llm = ChatOpenAI(model='gpt-3.5-turbo', api_key=apikey)

In [ ]:
#Define functions:
def get_employee_name(id: str) -> dict:
    if id == '123':
        return {'name': 'Smith', 'role': 'Manager', 'Department': 'IT'}
    elif id == '453':
        return {'name': 'Paul', 'role': 'Supervisor', 'Department': 'CCE'}
    else:
        return {}


In [ ]:
def check_leave_balance(id: str) -> dict:
    if id == '123':
        return {'remaining_days': '10', 'used_days': '30'}
    elif id == '453':
        return {'remaining_days': '2', 'used_days': '38'}
    else:
        return {}

In [ ]:
def generate_interview_question(role: str) -> list:
    if role == 'Computer Engineering':
        return [
            "Can you explain the differences between a process and a thread?",
            "How does a CPU handle interrupts, and why are they important?",
            "Describe how virtual memory works and why it's used.",
            "What are the key differences between RISC and CISC architectures?",
            "How would you optimize code for performance at the hardware level?"
        ]
    elif role == 'AI Engineering':
        return [
            "Explain the difference between supervised, unsupervised, and reinforcement learning.",
            "How do you handle imbalanced datasets in machine learning?",
            "What is the bias-variance tradeoff?",
            "Describe a project where you deployed a machine learning model in production.",
            "How would you select the right model for a classification problem?"
        ]
    elif role == 'Biomedical Engineering':
        return [
            "What biomedical devices have you worked with, and what challenges did you face?",
            "Explain how you would validate the accuracy and safety of a medical sensor.",
            "Describe a time you collaborated with clinicians or patients during a project.",
            "What is your experience with regulatory compliance in biomedical design?",
            "How would you improve the design of a prosthetic limb for better user comfort?"
        ]
    else:
        return ["No questions available for this role."]


🟢 Use with type hints (def check_leave_balance(id: str) -> dict: instead of def check_leave_balance(id) )for: 

Better code clarity

Fewer bugs

Improved editor support (autocomplete, linting)

In [ ]:
#Tools
from langchain.tools import Tool

emp_name_tool = Tool.from_function(
    name='check_employee_information',
    func=get_employee_name,
    description="Use this tool to retrieve an employee's name, role, and department by providing their employee ID."
)

leave_balance_tool = Tool.from_function(
    name='check_leave_balance',
    func=check_leave_balance,
    description='Use this tool to check how many leave days an employee has by providing the employee ID.'
)

interview_tool = Tool.from_function(
    name='generate_interview_questions',
    func=generate_interview_question,
    description='Use this tool to generate interview questions for a given job role.'
)


Tool key parameters:

1. name	: The name of the tool 
2. func:  The Python function to execute without ()
3. description:	A description of what the tool does (helps the agent decide)

Agent(decide what tool to use) parameters:

1. tools: A list of tools it can use

2. llm: The LLM (Language Model)  is used to understand the question and generate a human-like answer.

3. agent_type: The logic type (e.g., ZERO_SHOT_REACT_DESCRIPTION)

4. verbose: If True, shows step-by-step reasoning in the logs

In [ ]:
#Agent
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=[emp_name_tool, leave_balance_tool, interview_tool],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,#use the description to indicate which tool to be used
    verbose=True
)

How the Process Works:

User Query: The user asks a question (e.g., "What is my leave balance?").

LLM Processing:

The LLM inside the agent analyzes the query and understands that the user wants to know their leave balance.

The LLM decides whether to directly answer the question or call an external tool.

Tool Call (If Needed): If the LLM decides that a tool (e.g., check_leave_balance) is needed to fetch data, it formats the call and passes the necessary parameters (e.g., employee ID).

Tool Response: The tool executes (e.g., querying the database) and returns the required information (e.g., "You have 10 days of leave remaining").

LLM Response Generation:

The LLM formats the result into a clear and human-like answer.

For instance, "Your remaining leave balance is 10 days."

Final Answer: The agent sends the response back to the user.

In [ ]:
import streamlit as st 
st.set_page_config(page_title='HR-Assistant',layout='centered')
st.title('Hello Iam your HR assistnt')
question=st.text_input('Enter your question')
if question:
    response = agent.run(question)
    st.write(response)
else:
    st.write("Please enter a question.")

#Don't use context[0].message.content when you're using LangChain’s built-in tools, agents, or chains, because LangChain automatically handles the message processing for you.

✅ If the answer to a question is not available through any predefined functions (tools), and the answer needs to be generated from the LLM (like GPT) itself

langchain.llms: Use for simple one-time questions(from langchain.llms import OpenAI...OpenAI(model=,apikey=))

langchain-community.llms: Use for community or third-party LLMs (Hugging face)

langchain.chatmodels: Use for chat models that remember the conversation(fromlangchain.chatmodelsimport ChatOpenAi..... ChatOpenAI(model=,apikey=..) )

✅ Summary: How LangChain Extracts ID from Natural Language
When you type something like:

"Check for me leave balance from employee with id 123"

LangChain uses these components to understand and process your request:

🧠 1. LLM (Large Language Model)
Understands the intent ("check leave balance")

Recognizes that "123" is the employee ID, even without a special format

🤖 2. Agent
Decides which tool/function to use (e.g., check_leave_balance)

Based on the tool’s description, matches the query to the correct function

🛠️ 3. Tool Calling
LLM fills in the function arguments like this:

json
Copy
Edit
{
  "name": "check_leave_balance",
  "arguments": {
    "employee_id": "123"
  }
}

-------------------------------------------------------------------------------------------


1. Employee enters a question.

2. LLM analyzes and understands the question.

3. Agent decides if a tool needs to be used.

4. LLM extracts necessary parameters (like employee ID).

5. Agent selects the correct tool based on the question and tool descriptions.

6. Tool is called with extracted parameters.

7. Tool returns the result.

8. LLM uses the result to generate a natural language answer.

9. Assistant sends the final response to the employee.